# Tutorial Completo: FastAPI para Modelos de Machine Learning

## Introducción

En este tutorial aprenderás a crear una API REST completa usando **FastAPI** para servir modelos de Machine Learning. FastAPI es un framework moderno y rápido para construir APIs con Python 3.7+.

### ¿Qué vamos a construir?

Crearemos una aplicación que incluye:
1. **Endpoint básico** - Para verificar que la API funciona
2. **Endpoint de predicción** - Para hacer inferencias con nuestro modelo
3. **Endpoint de monitorización** - Para ver métricas y estado del modelo
4. **Endpoint de reentrenamiento** - Para actualizar el modelo con nuevos datos

### Requisitos previos

- Python 3.7+
- Conocimientos básicos de Machine Learning
- Familiaridad con APIs REST (recomendado)

---

## 1. Instalación de Dependencias

Primero necesitamos instalar todas las bibliotecas necesarias:

- **fastapi**: Framework para crear la API
- **uvicorn**: Servidor ASGI para ejecutar FastAPI
- **tensorflow**: Para crear y usar modelos de deep learning
- **numpy**: Para manipulación de arrays
- **pandas**: Para manejo de datos
- **scikit-learn**: Para métricas y preprocesamiento
- **pydantic**: Para validación de datos (incluido con FastAPI)

In [ ]:
# Instalación de todas las dependencias necesarias
# Ejecuta esta celda una sola vez
!pip install fastapi uvicorn tensorflow numpy pandas scikit-learn

## 2. Importación de Bibliotecas

Importamos todas las bibliotecas que usaremos en el tutorial.

In [1]:
# Importaciones para FastAPI y servidor
from fastapi import FastAPI, HTTPException, File, UploadFile
from pydantic import BaseModel, Field
import uvicorn

# Importaciones para Machine Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Importaciones para utilidades
import os
import json
from datetime import datetime
from typing import List, Dict, Any
import pickle

print("✓ Todas las bibliotecas importadas correctamente")
print(f"Versión de TensorFlow: {tf.__version__}")

✓ Todas las bibliotecas importadas correctamente
Versión de TensorFlow: 2.20.0


## 3. Creación y Guardado de un Modelo de Ejemplo

### 3.1 ¿Por qué necesitamos guardar modelos?

Cuando entrenamos un modelo de Machine Learning, queremos poder reutilizarlo sin tener que reentrenarlo cada vez. Keras ofrece dos formatos principales:

- **.keras** (recomendado): Formato nativo de Keras 3.0+, guarda todo en un solo archivo
- **.h5**: Formato HDF5, compatible con versiones anteriores

### 3.2 Creación de un dataset sintético

Para este tutorial, crearemos un problema de regresión simple: predecir el precio de una casa basándose en características como tamaño, número de habitaciones, etc.

In [2]:
# Configuración de semilla para reproducibilidad
np.random.seed(42)
tf.random.set_seed(42)

# Generación de datos sintéticos para predicción de precios de casas
# Características: tamaño (m2), habitaciones, baños, antigüedad (años)
n_samples = 1000

# Generamos características aleatorias
tamano = np.random.uniform(50, 300, n_samples)  # Tamaño entre 50 y 300 m2
habitaciones = np.random.randint(1, 6, n_samples)  # Entre 1 y 5 habitaciones
banos = np.random.randint(1, 4, n_samples)  # Entre 1 y 3 baños
antiguedad = np.random.uniform(0, 50, n_samples)  # Antigüedad entre 0 y 50 años

# Creamos la variable objetivo (precio) con una fórmula lógica + ruido
# Precio base: 1000€/m2, +20000€ por habitación, +15000€ por baño, -500€ por año de antigüedad
precio = (
    tamano * 1000 + 
    habitaciones * 20000 + 
    banos * 15000 - 
    antiguedad * 500 +
    np.random.normal(0, 20000, n_samples)  # Añadimos ruido
)

# Combinamos todas las características en una matriz
X = np.column_stack([tamano, habitaciones, banos, antiguedad])
y = precio

print(f"Dataset creado:")
print(f"  - Número de muestras: {n_samples}")
print(f"  - Número de características: {X.shape[1]}")
print(f"  - Rango de precios: {y.min():.0f}€ - {y.max():.0f}€")
print(f"\nPrimeras 5 muestras:")
print(f"{'Tamaño':<10} {'Habit.':<8} {'Baños':<8} {'Antigüedad':<12} {'Precio':<10}")
for i in range(5):
    print(f"{X[i,0]:<10.1f} {X[i,1]:<8.0f} {X[i,2]:<8.0f} {X[i,3]:<12.1f} {y[i]:<10.0f}€")

Dataset creado:
  - Número de muestras: 1000
  - Número de características: 4
  - Rango de precios: 32381€ - 449305€

Primeras 5 muestras:
Tamaño     Habit.   Baños    Antigüedad   Precio    
143.6      4        3        31.2         277667    €
287.7      3        2        28.5         397144    €
233.0      5        3        31.0         373740    €
199.7      1        2        10.1         227036    €
89.0       5        1        19.8         233867    €


### 3.3 Preparación de los datos

Antes de entrenar, debemos:
1. Dividir los datos en entrenamiento y prueba
2. Normalizar las características (importante para redes neuronales)

In [3]:
# División de datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,  # 20% para prueba
    random_state=42  # Para reproducibilidad
)

# Normalización de características usando StandardScaler
# Esto convierte los datos para que tengan media=0 y desviación estándar=1
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Ajustamos y transformamos training
X_test_scaled = scaler.transform(X_test)  # Solo transformamos test (sin fit)

# Guardamos el scaler para usarlo después en las predicciones
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print(f"✓ Datos divididos:")
print(f"  - Training set: {X_train.shape[0]} muestras")
print(f"  - Test set: {X_test.shape[0]} muestras")
print(f"\n✓ Scaler guardado en 'scaler.pkl'")

✓ Datos divididos:
  - Training set: 800 muestras
  - Test set: 200 muestras

✓ Scaler guardado en 'scaler.pkl'


### 3.4 Creación del Modelo

Crearemos una red neuronal simple con:
- Capa de entrada: 4 características
- 2 capas ocultas con activación ReLU
- Capa de salida: 1 neurona (predicción del precio)

In [4]:
# Construcción del modelo usando la API Sequential de Keras
modelo = models.Sequential([
    # Capa de entrada: especificamos el número de características
    layers.Input(shape=(4,)),  # 4 características
    
    # Primera capa oculta: 64 neuronas con activación ReLU
    layers.Dense(64, activation='relu', name='capa_oculta_1'),
    
    # Segunda capa oculta: 32 neuronas con activación ReLU
    layers.Dense(32, activation='relu', name='capa_oculta_2'),
    
    # Capa de salida: 1 neurona (predicción del precio)
    # Sin activación porque es un problema de regresión
    layers.Dense(1, name='salida')
])

# Compilación del modelo
# - Optimizer: Adam (buen optimizer por defecto)
# - Loss: MSE (Mean Squared Error) para regresión
# - Metrics: MAE (Mean Absolute Error) para seguimiento
modelo.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

# Mostramos la arquitectura del modelo
print("Arquitectura del modelo:\n")
modelo.summary()

Arquitectura del modelo:



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ capa_oculta_1 (Dense)           │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa_oculta_2 (Dense)           │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ salida (Dense)                  │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,433 (9.50 KB)

 Trainable params: 2,433 (9.50 KB)

 Non-trainable params: 0 (0.00 B)

### 3.5 Entrenamiento del Modelo

In [5]:
# Entrenamiento del modelo
print("Entrenando el modelo...\n")

history = modelo.fit(
    X_train_scaled,  # Datos de entrada normalizados
    y_train,  # Precios objetivo
    epochs=100,  # Número de épocas
    batch_size=32,  # Tamaño del batch
    validation_split=0.2,  # 20% de training para validación
    verbose=1  # Mostrar progreso
)

print("\n✓ Entrenamiento completado")

Entrenando el modelo...

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 69756403712.0000 - mean_absolute_error: 251284.2500 - val_loss: 71279558656.0000 - val_mean_absolute_error: 253447.0938
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 69755838464.0000 - mean_absolute_error: 251283.1250 - val_loss: 71278862336.0000 - val_mean_absolute_error: 253445.7500
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 69754945536.0000 - mean_absolute_error: 251281.4062 - val_loss: 71277740032.0000 - val_mean_absolute_error: 253443.5469
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 69753520128.0000 - mean_absolute_error: 251278.6562 - val_loss: 71275970560.0000 - val_mean_absolute_error: 253440.1562
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 69751316480.0000 - mean_absolute_error: 251274.3438 - val_loss: 71273250816.0000 - val_mean_absolute_error: 253434.9531
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 69747965952.0000 - me

### 3.6 Evaluación del Modelo

In [6]:
# Evaluación en el conjunto de prueba
test_loss, test_mae = modelo.evaluate(X_test_scaled, y_test, verbose=0)

# Hacemos predicciones para calcular más métricas
y_pred = modelo.predict(X_test_scaled, verbose=0)

# Calculamos métricas adicionales
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Métricas en el conjunto de prueba:")
print(f"  - MSE (Mean Squared Error): {mse:,.0f}")
print(f"  - RMSE (Root Mean Squared Error): {rmse:,.0f}€")
print(f"  - MAE (Mean Absolute Error): {mae:,.0f}€")
print(f"  - R² Score: {r2:.4f}")
print(f"\nInterpretación: En promedio, nuestras predicciones se desvían ±{mae:,.0f}€ del precio real")

Métricas en el conjunto de prueba:
  - MSE (Mean Squared Error): 37,153,426,918
  - RMSE (Root Mean Squared Error): 192,752€
  - MAE (Mean Absolute Error): 187,355€
  - R² Score: -5.2438

Interpretación: En promedio, nuestras predicciones se desvían ±187,355€ del precio real


### 3.7 Guardado del Modelo

Ahora guardaremos el modelo en ambos formatos para que puedas ver cómo funciona cada uno:

#### Formato .keras (Recomendado)
- Formato nativo y moderno
- Guarda todo: arquitectura, pesos, optimizer, etc.
- Más eficiente y rápido

#### Formato .h5 (Legado)
- Formato HDF5
- Compatible con versiones anteriores de Keras
- Más común en proyectos antiguos

In [7]:
# Crear carpeta para modelos si no existe
os.makedirs('modelos', exist_ok=True)

# 1. Guardar en formato .keras (RECOMENDADO)
modelo.save('modelos/modelo_precios.keras')
print("✓ Modelo guardado en formato .keras")

# 2. Guardar en formato .h5 (compatibilidad)
modelo.save('modelos/modelo_precios.h5')
print("✓ Modelo guardado en formato .h5")

# 3. Guardar metadatos del modelo (útil para monitorización)
metadata = {
    'fecha_entrenamiento': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'metricas': {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    },
    'arquitectura': {
        'capas': len(modelo.layers),
        'parametros_totales': modelo.count_params()
    },
    'datos_entrenamiento': {
        'n_muestras_train': len(X_train),
        'n_muestras_test': len(X_test),
        'n_caracteristicas': X_train.shape[1]
    }
}

with open('modelos/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print("✓ Metadatos guardados en 'modelos/metadata.json'")

print("\n" + "="*50)
print("ARCHIVOS CREADOS:")
print("  📁 modelos/")
print("    📄 modelo_precios.keras  (formato recomendado)")
print("    📄 modelo_precios.h5     (formato legado)")
print("    📄 metadata.json         (información del modelo)")
print("  📄 scaler.pkl              (normalizador de datos)")
print("="*50)

✓ Modelo guardado en formato .keras
✓ Modelo guardado en formato .h5
✓ Metadatos guardados en 'modelos/metadata.json'

ARCHIVOS CREADOS:
  📁 modelos/
    📄 modelo_precios.keras  (formato recomendado)
    📄 modelo_precios.h5     (formato legado)
    📄 metadata.json         (información del modelo)
  📄 scaler.pkl              (normalizador de datos)


### 3.8 Verificación: Carga del Modelo

Vamos a verificar que podemos cargar correctamente el modelo guardado:

In [8]:
# Cargar modelo desde formato .keras
modelo_cargado_keras = keras.models.load_model('modelos/modelo_precios.keras')
print("✓ Modelo .keras cargado correctamente")

# Cargar modelo desde formato .h5
modelo_cargado_h5 = keras.models.load_model('modelos/modelo_precios.h5')
print("✓ Modelo .h5 cargado correctamente")

# Verificar que las predicciones son idénticas
pred_original = modelo.predict(X_test_scaled[:5], verbose=0)
pred_keras = modelo_cargado_keras.predict(X_test_scaled[:5], verbose=0)
pred_h5 = modelo_cargado_h5.predict(X_test_scaled[:5], verbose=0)

print("\nComparación de predicciones (primeras 5 muestras):")
print(f"{'Original':<15} {'Cargado .keras':<15} {'Cargado .h5':<15} {'Real':<15}")
for i in range(5):
    print(f"{pred_original[i][0]:<15.0f} {pred_keras[i][0]:<15.0f} {pred_h5[i][0]:<15.0f} {y_test.iloc[i] if isinstance(y_test, pd.Series) else y_test[i]:<15.0f}")

print("\n✓ Verificación completada: Los modelos cargados funcionan correctamente")

✓ Modelo .keras cargado correctamente
✓ Modelo .h5 cargado correctamente

Comparación de predicciones (primeras 5 muestras):
Original        Cargado .keras  Cargado .h5     Real           
19420           19420           19420           200758         
78198           78198           78198           301970         
75496           75496           75496           273995         
70698           70698           70698           220921         
151371          151371          151371          359485         

✓ Verificación completada: Los modelos cargados funcionan correctamente


---

## 4. Creación de la API con FastAPI

Ahora que tenemos nuestro modelo guardado, vamos a crear la API REST.

### 4.1 Modelos de Datos con Pydantic

Pydantic nos permite definir esquemas de datos con validación automática. Esto asegura que los datos que recibe nuestra API sean correctos.

In [9]:
# Modelos de datos para validación de requests/responses

# Modelo para recibir datos de predicción
class DatosCasa(BaseModel):
    """Datos de entrada para predecir el precio de una casa"""
    tamano: float = Field(..., description="Tamaño en m2", gt=0, example=150.0)
    habitaciones: int = Field(..., description="Número de habitaciones", ge=1, le=10, example=3)
    banos: int = Field(..., description="Número de baños", ge=1, le=5, example=2)
    antiguedad: float = Field(..., description="Antigüedad en años", ge=0, le=100, example=10.0)
    
    class Config:
        json_schema_extra = {
            "example": {
                "tamano": 150.0,
                "habitaciones": 3,
                "banos": 2,
                "antiguedad": 10.0
            }
        }

# Modelo para respuesta de predicción
class PrediccionRespuesta(BaseModel):
    """Respuesta con la predicción del precio"""
    precio_predicho: float = Field(..., description="Precio predicho en euros")
    confianza: str = Field(..., description="Nivel de confianza de la predicción")
    timestamp: str = Field(..., description="Fecha y hora de la predicción")

# Modelo para datos de reentrenamiento
class DatosReentrenamiento(BaseModel):
    """Datos para reentrenar el modelo"""
    datos: List[DatosCasa] = Field(..., description="Lista de datos de casas")
    precios: List[float] = Field(..., description="Precios reales correspondientes")
    epochs: int = Field(default=50, description="Número de épocas para reentrenamiento", ge=1, le=200)

print("✓ Modelos de datos definidos correctamente")

✓ Modelos de datos definidos correctamente


C:\Users\borja\AppData\Local\Temp\ipykernel_27396\3911377819.py:6: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  tamano: float = Field(..., description="Tamaño en m2", gt=0, example=150.0)
C:\Users\borja\AppData\Local\Temp\ipykernel_27396\3911377819.py:7: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  habitaciones: int = Field(..., description="Número de habitaciones", ge=1, le=10, example=3)
C:\Users\borja\AppData\Local\Temp\ipykernel_27396\3911377819.py:8: PydanticDeprecatedSince20: Using extra keywo

### 4.2 Variables Globales y Carga Inicial

Definimos variables globales para mantener el estado del modelo, métricas y scaler.

In [10]:
# Variables globales para la aplicación
modelo_actual = None  # Modelo cargado en memoria
scaler_actual = None  # Scaler para normalización
metadata_actual = {}  # Metadatos del modelo
historial_predicciones = []  # Historial para monitorización

# Función para cargar el modelo y sus componentes
def cargar_modelo():
    """
    Carga el modelo, scaler y metadata desde disco.
    Esta función se ejecuta al iniciar la aplicación.
    """
    global modelo_actual, scaler_actual, metadata_actual
    
    try:
        # Cargar el modelo (intentamos .keras primero, luego .h5)
        if os.path.exists('modelos/modelo_precios.keras'):
            modelo_actual = keras.models.load_model('modelos/modelo_precios.keras')
            print("✓ Modelo .keras cargado")
        elif os.path.exists('modelos/modelo_precios.h5'):
            modelo_actual = keras.models.load_model('modelos/modelo_precios.h5')
            print("✓ Modelo .h5 cargado")
        else:
            raise FileNotFoundError("No se encontró ningún modelo guardado")
        
        # Cargar el scaler
        with open('scaler.pkl', 'rb') as f:
            scaler_actual = pickle.load(f)
        print("✓ Scaler cargado")
        
        # Cargar metadata si existe
        if os.path.exists('modelos/metadata.json'):
            with open('modelos/metadata.json', 'r') as f:
                metadata_actual = json.load(f)
            print("✓ Metadata cargada")
        
        return True
    except Exception as e:
        print(f"❌ Error al cargar el modelo: {e}")
        return False

# Cargar el modelo al inicio
if cargar_modelo():
    print("\n✓ Sistema listo para usar")
else:
    print("\n⚠️ El sistema no está completamente inicializado")

✓ Modelo .keras cargado
✓ Scaler cargado
✓ Metadata cargada

✓ Sistema listo para usar


---

## 5. ENDPOINT 1: Endpoint Básico de Saludo

Este es el endpoint más simple. Sirve para:
- Verificar que la API está funcionando
- Health check básico
- Prueba de conectividad

In [11]:
# Creamos la aplicación FastAPI
app = FastAPI(
    title="API de Predicción de Precios de Casas",
    description="API REST para servir modelos de ML con FastAPI",
    version="1.0.0"
)

# ENDPOINT 1: Ruta raíz - Saludo básico
@app.get("/", tags=["General"])
async def raiz():
    """
    Endpoint básico de bienvenida.
    
    Returns:
        dict: Mensaje de bienvenida y estado del sistema
    """
    return {
        "mensaje": "¡Hola! Bienvenido a la API de predicción de precios de casas",
        "version": "1.0.0",
        "status": "operativo",
        "modelo_cargado": modelo_actual is not None,
        "endpoints_disponibles": [
            "/docs - Documentación interactiva",
            "/predecir - Hacer predicciones",
            "/monitorizar - Ver métricas del modelo",
            "/reentrenar - Reentrenar el modelo"
        ]
    }

# ENDPOINT 1.1: Health check
@app.get("/health", tags=["General"])
async def health_check():
    """
    Verifica el estado de salud de la API.
    
    Returns:
        dict: Estado detallado del sistema
    """
    # Verificamos todos los componentes
    estado = {
        "status": "healthy",
        "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "componentes": {
            "modelo": "OK" if modelo_actual is not None else "ERROR",
            "scaler": "OK" if scaler_actual is not None else "ERROR",
            "metadata": "OK" if metadata_actual else "WARNING"
        },
        "estadisticas": {
            "predicciones_realizadas": len(historial_predicciones),
            "fecha_ultimo_entrenamiento": metadata_actual.get('fecha_entrenamiento', 'Desconocida')
        }
    }
    
    # Si algún componente crítico falla, cambiamos el estado
    if modelo_actual is None or scaler_actual is None:
        estado["status"] = "unhealthy"
        raise HTTPException(status_code=503, detail="Servicio no disponible")
    
    return estado

print("✓ Endpoints básicos creados")
print("  - GET /")
print("  - GET /health")

✓ Endpoints básicos creados
  - GET /
  - GET /health


---

## 6. ENDPOINT 2: Predicción con el Modelo

Este es el endpoint principal de la API. Permite hacer predicciones usando el modelo cargado.

### Flujo de predicción:
1. Recibir datos de entrada (validados por Pydantic)
2. Normalizar los datos con el scaler
3. Hacer la predicción con el modelo
4. Guardar en el historial para monitorización
5. Retornar el resultado

In [12]:
# ENDPOINT 2: Predicción
@app.post("/predecir", response_model=PrediccionRespuesta, tags=["Predicción"])
async def predecir_precio(datos: DatosCasa):
    """
    Predice el precio de una casa basándose en sus características.
    
    Args:
        datos (DatosCasa): Características de la casa
        
    Returns:
        PrediccionRespuesta: Precio predicho y metadatos
        
    Raises:
        HTTPException: Si el modelo no está cargado o hay un error en la predicción
    """
    # Verificar que el modelo esté cargado
    if modelo_actual is None or scaler_actual is None:
        raise HTTPException(
            status_code=503,
            detail="El modelo no está cargado. Por favor, reinicia el servidor."
        )
    
    try:
        # Paso 1: Preparar los datos de entrada
        # Convertimos los datos de Pydantic a un array numpy
        entrada = np.array([[
            datos.tamano,
            datos.habitaciones,
            datos.banos,
            datos.antiguedad
        ]])
        
        # Paso 2: Normalizar los datos usando el scaler
        # Es CRÍTICO usar el mismo scaler que se usó en el entrenamiento
        entrada_normalizada = scaler_actual.transform(entrada)
        
        # Paso 3: Hacer la predicción
        prediccion = modelo_actual.predict(entrada_normalizada, verbose=0)
        precio_predicho = float(prediccion[0][0])
        
        # Paso 4: Calcular nivel de confianza (simplificado)
        # En un sistema real, usaríamos intervalos de confianza o predicción
        # Aquí usamos una heurística basada en el MAE del modelo
        mae_modelo = metadata_actual.get('metricas', {}).get('mae', 20000)
        error_relativo = (mae_modelo / precio_predicho) * 100
        
        if error_relativo < 10:
            confianza = "alta"
        elif error_relativo < 20:
            confianza = "media"
        else:
            confianza = "baja"
        
        # Paso 5: Guardar en historial para monitorización
        historial_predicciones.append({
            "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "entrada": {
                "tamano": datos.tamano,
                "habitaciones": datos.habitaciones,
                "banos": datos.banos,
                "antiguedad": datos.antiguedad
            },
            "prediccion": precio_predicho,
            "confianza": confianza
        })
        
        # Limitar el historial a las últimas 1000 predicciones
        if len(historial_predicciones) > 1000:
            historial_predicciones.pop(0)
        
        # Paso 6: Retornar la respuesta
        return PrediccionRespuesta(
            precio_predicho=round(precio_predicho, 2),
            confianza=confianza,
            timestamp=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        )
        
    except Exception as e:
        # Manejo de errores
        raise HTTPException(
            status_code=500,
            detail=f"Error al realizar la predicción: {str(e)}"
        )

# ENDPOINT 2.1: Predicción por lotes (batch prediction)
@app.post("/predecir/lote", tags=["Predicción"])
async def predecir_lote(datos_lista: List[DatosCasa]):
    """
    Predice precios para múltiples casas a la vez.
    Más eficiente que hacer múltiples llamadas individuales.
    
    Args:
        datos_lista (List[DatosCasa]): Lista de casas
        
    Returns:
        dict: Lista de predicciones
    """
    if modelo_actual is None or scaler_actual is None:
        raise HTTPException(status_code=503, detail="Modelo no disponible")
    
    try:
        # Convertir todos los datos a un array numpy
        entrada = np.array([
            [d.tamano, d.habitaciones, d.banos, d.antiguedad]
            for d in datos_lista
        ])
        
        # Normalizar y predecir
        entrada_normalizada = scaler_actual.transform(entrada)
        predicciones = modelo_actual.predict(entrada_normalizada, verbose=0)
        
        # Formatear resultados
        resultados = [
            {
                "indice": i,
                "entrada": {
                    "tamano": datos_lista[i].tamano,
                    "habitaciones": datos_lista[i].habitaciones,
                    "banos": datos_lista[i].banos,
                    "antiguedad": datos_lista[i].antiguedad
                },
                "precio_predicho": round(float(predicciones[i][0]), 2)
            }
            for i in range(len(predicciones))
        ]
        
        return {
            "n_predicciones": len(resultados),
            "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "predicciones": resultados
        }
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error: {str(e)}")

print("✓ Endpoints de predicción creados")
print("  - POST /predecir")
print("  - POST /predecir/lote")

✓ Endpoints de predicción creados
  - POST /predecir
  - POST /predecir/lote


---

## 7. ENDPOINT 3: Monitorización del Modelo

La monitorización es crucial en producción. Este endpoint permite:
- Ver métricas del modelo actual
- Analizar el historial de predicciones
- Detectar posibles problemas o degradación del modelo

In [13]:
# ENDPOINT 3: Monitorización
@app.get("/monitorizar", tags=["Monitorización"])
async def monitorizar_modelo():
    """
    Devuelve métricas y estadísticas sobre el modelo en producción.
    
    Incluye:
    - Métricas de entrenamiento
    - Estadísticas de uso
    - Información de la arquitectura
    - Análisis del historial de predicciones
    
    Returns:
        dict: Métricas completas del modelo
    """
    if modelo_actual is None:
        raise HTTPException(status_code=503, detail="Modelo no disponible")
    
    # 1. Información básica del modelo
    info_basica = {
        "nombre": "Modelo de Predicción de Precios de Casas",
        "version": "1.0.0",
        "estado": "activo",
        "fecha_carga": metadata_actual.get('fecha_entrenamiento', 'Desconocida')
    }
    
    # 2. Métricas de entrenamiento (del archivo metadata.json)
    metricas_entrenamiento = metadata_actual.get('metricas', {
        "mse": "No disponible",
        "rmse": "No disponible",
        "mae": "No disponible",
        "r2": "No disponible"
    })
    
    # 3. Información de la arquitectura
    arquitectura = {
        "n_capas": len(modelo_actual.layers),
        "parametros_totales": modelo_actual.count_params(),
        "parametros_entrenables": sum([tf.size(w).numpy() for w in modelo_actual.trainable_weights]),
        "capas_detalle": [
            {
                "nombre": layer.name,
                "tipo": layer.__class__.__name__,
                "forma_salida": str(layer.output_shape)
            }
            for layer in modelo_actual.layers
        ]
    }
    
    # 4. Estadísticas de uso (desde el inicio del servidor)
    estadisticas_uso = {
        "total_predicciones": len(historial_predicciones),
        "predicciones_ultima_hora": sum(
            1 for p in historial_predicciones
            if (datetime.now() - datetime.strptime(p['timestamp'], '%Y-%m-%d %H:%M:%S')).seconds < 3600
        ) if historial_predicciones else 0
    }
    
    # 5. Análisis del historial de predicciones
    if historial_predicciones:
        precios_predichos = [p['prediccion'] for p in historial_predicciones]
        distribucion_confianza = {
            "alta": sum(1 for p in historial_predicciones if p['confianza'] == 'alta'),
            "media": sum(1 for p in historial_predicciones if p['confianza'] == 'media'),
            "baja": sum(1 for p in historial_predicciones if p['confianza'] == 'baja')
        }
        
        analisis_predicciones = {
            "precio_promedio": round(np.mean(precios_predichos), 2),
            "precio_mediano": round(np.median(precios_predichos), 2),
            "precio_min": round(np.min(precios_predichos), 2),
            "precio_max": round(np.max(precios_predichos), 2),
            "desviacion_estandar": round(np.std(precios_predichos), 2),
            "distribucion_confianza": distribucion_confianza
        }
    else:
        analisis_predicciones = {
            "mensaje": "No hay predicciones registradas aún"
        }
    
    # 6. Recomendaciones y alertas
    recomendaciones = []
    
    # Alerta si hay muchas predicciones de baja confianza
    if historial_predicciones:
        pct_baja_confianza = (distribucion_confianza.get('baja', 0) / len(historial_predicciones)) * 100
        if pct_baja_confianza > 30:
            recomendaciones.append({
                "tipo": "warning",
                "mensaje": f"Alto porcentaje de predicciones con baja confianza ({pct_baja_confianza:.1f}%). Considera reentrenar el modelo."
            })
    
    # Alerta si el modelo es antiguo (ejemplo: más de 30 días)
    # En producción real, compararías con la fecha actual
    recomendaciones.append({
        "tipo": "info",
        "mensaje": "Recomendación: Reentrena el modelo periódicamente con datos nuevos para mantener su precisión."
    })
    
    # 7. Retornar todo el análisis
    return {
        "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "info_basica": info_basica,
        "metricas_entrenamiento": metricas_entrenamiento,
        "arquitectura": arquitectura,
        "estadisticas_uso": estadisticas_uso,
        "analisis_predicciones": analisis_predicciones,
        "recomendaciones": recomendaciones
    }

# ENDPOINT 3.1: Historial de predicciones
@app.get("/monitorizar/historial", tags=["Monitorización"])
async def obtener_historial(limite: int = 100):
    """
    Devuelve las últimas N predicciones realizadas.
    
    Args:
        limite (int): Número máximo de predicciones a devolver (default: 100)
        
    Returns:
        dict: Historial de predicciones
    """
    # Limitar el número de resultados
    limite = min(limite, 1000)  # Máximo 1000 predicciones
    
    return {
        "total_predicciones": len(historial_predicciones),
        "limite": limite,
        "predicciones": historial_predicciones[-limite:]  # Últimas N predicciones
    }

print("✓ Endpoints de monitorización creados")
print("  - GET /monitorizar")
print("  - GET /monitorizar/historial")

✓ Endpoints de monitorización creados
  - GET /monitorizar
  - GET /monitorizar/historial


---

## 8. ENDPOINT 4: Reentrenamiento del Modelo

El reentrenamiento permite actualizar el modelo con nuevos datos sin tener que reconstruir toda la API.

### Consideraciones importantes:
- En producción, esto se haría de forma asíncrona (usando Celery, por ejemplo)
- Se debería validar la calidad de los nuevos datos
- Es recomendable hacer validación cruzada antes de actualizar el modelo en producción

In [14]:
# ENDPOINT 4: Reentrenamiento
@app.post("/reentrenar", tags=["Reentrenamiento"])
async def reentrenar_modelo(datos_entrenamiento: DatosReentrenamiento):
    """
    Reentrena el modelo con nuevos datos.
    
    IMPORTANTE: En producción, este proceso debería ser asíncrono y
    realizarse en un worker separado (ej: Celery) para no bloquear la API.
    
    Args:
        datos_entrenamiento (DatosReentrenamiento): Nuevos datos para entrenar
        
    Returns:
        dict: Resultado del reentrenamiento con métricas
        
    Raises:
        HTTPException: Si hay errores en los datos o el entrenamiento
    """
    global modelo_actual, metadata_actual
    
    if modelo_actual is None or scaler_actual is None:
        raise HTTPException(status_code=503, detail="Modelo no disponible")
    
    try:
        # 1. Validar que tenemos suficientes datos
        if len(datos_entrenamiento.datos) < 10:
            raise HTTPException(
                status_code=400,
                detail="Se necesitan al menos 10 muestras para reentrenar"
            )
        
        # 2. Validar que el número de datos coincide con el número de precios
        if len(datos_entrenamiento.datos) != len(datos_entrenamiento.precios):
            raise HTTPException(
                status_code=400,
                detail="El número de datos debe coincidir con el número de precios"
            )
        
        # 3. Preparar los datos de entrenamiento
        # Convertir los datos de Pydantic a arrays numpy
        X_nuevo = np.array([
            [d.tamano, d.habitaciones, d.banos, d.antiguedad]
            for d in datos_entrenamiento.datos
        ])
        y_nuevo = np.array(datos_entrenamiento.precios)
        
        # 4. Dividir en train/test para validación
        X_train_nuevo, X_test_nuevo, y_train_nuevo, y_test_nuevo = train_test_split(
            X_nuevo, y_nuevo,
            test_size=0.2,
            random_state=42
        )
        
        # 5. Normalizar los datos usando el scaler existente
        # IMPORTANTE: Usamos transform(), NO fit_transform()
        # para mantener la misma escala que el modelo original
        X_train_scaled = scaler_actual.transform(X_train_nuevo)
        X_test_scaled = scaler_actual.transform(X_test_nuevo)
        
        # 6. Guardar métricas del modelo antes del reentrenamiento
        metricas_antes = metadata_actual.get('metricas', {})
        
        # 7. Reentrenar el modelo
        # Usamos el modelo existente (transfer learning)
        print(f"Iniciando reentrenamiento con {len(X_train_nuevo)} muestras...")
        
        history = modelo_actual.fit(
            X_train_scaled,
            y_train_nuevo,
            epochs=datos_entrenamiento.epochs,
            batch_size=min(32, len(X_train_nuevo) // 4),  # Ajustamos batch size
            validation_split=0.2,
            verbose=0  # Silencioso para no saturar logs
        )
        
        # 8. Evaluar el modelo reentrenado
        y_pred = modelo_actual.predict(X_test_scaled, verbose=0)
        
        # Calcular métricas
        mse_nuevo = mean_squared_error(y_test_nuevo, y_pred)
        rmse_nuevo = np.sqrt(mse_nuevo)
        mae_nuevo = mean_absolute_error(y_test_nuevo, y_pred)
        r2_nuevo = r2_score(y_test_nuevo, y_pred)
        
        metricas_despues = {
            'mse': float(mse_nuevo),
            'rmse': float(rmse_nuevo),
            'mae': float(mae_nuevo),
            'r2': float(r2_nuevo)
        }
        
        # 9. Actualizar metadata
        metadata_actual['fecha_entrenamiento'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        metadata_actual['metricas'] = metricas_despues
        metadata_actual['datos_entrenamiento'] = {
            'n_muestras_train': len(X_train_nuevo),
            'n_muestras_test': len(X_test_nuevo),
            'n_caracteristicas': X_nuevo.shape[1]
        }
        
        # 10. Guardar el modelo reentrenado
        modelo_actual.save('modelos/modelo_precios.keras')
        with open('modelos/metadata.json', 'w') as f:
            json.dump(metadata_actual, f, indent=2)
        
        # 11. Preparar respuesta con comparación
        return {
            "status": "exitoso",
            "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "datos_utilizados": {
                "total_muestras": len(X_nuevo),
                "muestras_entrenamiento": len(X_train_nuevo),
                "muestras_test": len(X_test_nuevo),
                "epochs": datos_entrenamiento.epochs
            },
            "metricas_antes": metricas_antes,
            "metricas_despues": metricas_despues,
            "mejora": {
                "mae": f"{((metricas_antes.get('mae', 0) - mae_nuevo) / metricas_antes.get('mae', 1)) * 100:.2f}%" if metricas_antes.get('mae') else "N/A",
                "r2": f"{((r2_nuevo - metricas_antes.get('r2', 0)) / metricas_antes.get('r2', 1)) * 100:.2f}%" if metricas_antes.get('r2') else "N/A"
            },
            "modelo_guardado": "modelos/modelo_precios.keras",
            "mensaje": "Modelo reentrenado y guardado exitosamente"
        }
        
    except HTTPException:
        raise  # Re-lanzar excepciones HTTP
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Error durante el reentrenamiento: {str(e)}"
        )

# ENDPOINT 4.1: Validar datos antes de reentrenar
@app.post("/reentrenar/validar", tags=["Reentrenamiento"])
async def validar_datos_reentrenamiento(datos_entrenamiento: DatosReentrenamiento):
    """
    Valida los datos de reentrenamiento sin ejecutar el entrenamiento.
    Útil para verificar que los datos son correctos antes de reentrenar.
    
    Args:
        datos_entrenamiento (DatosReentrenamiento): Datos a validar
        
    Returns:
        dict: Resultado de la validación
    """
    # Validaciones básicas
    problemas = []
    advertencias = []
    
    # Número de muestras
    if len(datos_entrenamiento.datos) < 10:
        problemas.append("Se necesitan al menos 10 muestras (recomendado: 100+)")
    elif len(datos_entrenamiento.datos) < 50:
        advertencias.append("Se recomienda al menos 50 muestras para un buen reentrenamiento")
    
    # Concordancia de datos y precios
    if len(datos_entrenamiento.datos) != len(datos_entrenamiento.precios):
        problemas.append(f"Discordancia: {len(datos_entrenamiento.datos)} datos vs {len(datos_entrenamiento.precios)} precios")
    
    # Validar rangos de valores
    precios = datos_entrenamiento.precios
    if any(p <= 0 for p in precios):
        problemas.append("Hay precios negativos o cero")
    
    # Estadísticas de los datos
    if precios:
        estadisticas = {
            "precio_promedio": np.mean(precios),
            "precio_min": np.min(precios),
            "precio_max": np.max(precios),
            "desviacion_estandar": np.std(precios)
        }
    else:
        estadisticas = {}
    
    # Resultado de validación
    es_valido = len(problemas) == 0
    
    return {
        "valido": es_valido,
        "n_muestras": len(datos_entrenamiento.datos),
        "n_precios": len(datos_entrenamiento.precios),
        "problemas": problemas,
        "advertencias": advertencias,
        "estadisticas": estadisticas,
        "mensaje": "Datos válidos para reentrenamiento" if es_valido else "Corrige los problemas antes de reentrenar"
    }

print("✓ Endpoints de reentrenamiento creados")
print("  - POST /reentrenar")
print("  - POST /reentrenar/validar")

✓ Endpoints de reentrenamiento creados
  - POST /reentrenar
  - POST /reentrenar/validar


---

## 9. Ejecución del Servidor

### Opción 1: Ejecutar desde el notebook (para desarrollo)

**IMPORTANTE**: Si ejecutas el servidor desde Jupyter, la celda quedará bloqueada mientras el servidor esté corriendo. Para detenerlo, usa el botón de "stop" del notebook.

### Opción 2: Ejecutar desde terminal (recomendado para producción)

```bash
uvicorn nombre_archivo:app --reload --host 0.0.0.0 --port 8000
```

Donde:
- `nombre_archivo` es el nombre de tu archivo Python (sin .py)
- `--reload`: Reinicia automáticamente cuando cambias código (solo desarrollo)
- `--host 0.0.0.0`: Permite acceso desde cualquier IP
- `--port 8000`: Puerto donde escucha el servidor

In [ ]:
# NOTA: Esta celda iniciará el servidor. 
# Para detenerlo en Jupyter, usa el botón de "stop" o interrumpe el kernel

# Cargar el modelo antes de iniciar el servidor
print("Inicializando servidor...")
cargar_modelo()

# Configuración del servidor
# Para desarrollo local:
# - host="127.0.0.1" = solo accesible desde tu máquina
# - host="0.0.0.0" = accesible desde otras máquinas en la red
config = uvicorn.Config(
    app,
    host="127.0.0.1",  # Cambia a "0.0.0.0" para acceso externo
    port=8000,
    log_level="info"
)

server = uvicorn.Server(config)

print("\n" + "="*60)
print("🚀 SERVIDOR FASTAPI INICIADO")
print("="*60)
print(f"📍 URL: http://127.0.0.1:8000")
print(f"📚 Documentación interactiva: http://127.0.0.1:8000/docs")
print(f"📖 Documentación alternativa: http://127.0.0.1:8000/redoc")
print("="*60)
print("\nPara detener el servidor: Interrumpe el kernel o presiona el botón 'stop'\n")

# Iniciar el servidor
# NOTA: Esta línea bloqueará la ejecución hasta que detengas el servidor
await server.serve()

---

## 10. Testing de la API

### 10.1 Testing con Python (requests)

Una vez que el servidor esté corriendo, puedes probar los endpoints desde otra terminal o notebook.

In [ ]:
# IMPORTANTE: Ejecuta esta celda SOLO si el servidor está corriendo en otra terminal o celda
import requests

# URL base de la API
BASE_URL = "http://127.0.0.1:8000"

print("\n" + "="*60)
print("PROBANDO ENDPOINTS DE LA API")
print("="*60 + "\n")

# 1. Test del endpoint raíz
print("1️⃣ GET / (Endpoint de bienvenida)")
print("-" * 40)
response = requests.get(f"{BASE_URL}/")
print(f"Status Code: {response.status_code}")
print(f"Respuesta: {response.json()}")
print()

# 2. Test del health check
print("2️⃣ GET /health (Health check)")
print("-" * 40)
response = requests.get(f"{BASE_URL}/health")
print(f"Status Code: {response.status_code}")
print(f"Respuesta: {json.dumps(response.json(), indent=2)}")
print()

# 3. Test de predicción individual
print("3️⃣ POST /predecir (Predicción individual)")
print("-" * 40)
datos_casa = {
    "tamano": 120.0,
    "habitaciones": 3,
    "banos": 2,
    "antiguedad": 5.0
}
print(f"Datos enviados: {datos_casa}")
response = requests.post(f"{BASE_URL}/predecir", json=datos_casa)
print(f"Status Code: {response.status_code}")
print(f"Respuesta: {json.dumps(response.json(), indent=2)}")
print()

# 4. Test de predicción por lotes
print("4️⃣ POST /predecir/lote (Predicción por lotes)")
print("-" * 40)
datos_lote = [
    {"tamano": 100.0, "habitaciones": 2, "banos": 1, "antiguedad": 10.0},
    {"tamano": 150.0, "habitaciones": 3, "banos": 2, "antiguedad": 5.0},
    {"tamano": 200.0, "habitaciones": 4, "banos": 3, "antiguedad": 2.0}
]
print(f"Número de casas: {len(datos_lote)}")
response = requests.post(f"{BASE_URL}/predecir/lote", json=datos_lote)
print(f"Status Code: {response.status_code}")
resultado = response.json()
print(f"Predicciones obtenidas: {resultado['n_predicciones']}")
for pred in resultado['predicciones']:
    print(f"  Casa {pred['indice']}: {pred['precio_predicho']:.2f}€")
print()

# 5. Test de monitorización
print("5️⃣ GET /monitorizar (Monitorización)")
print("-" * 40)
response = requests.get(f"{BASE_URL}/monitorizar")
print(f"Status Code: {response.status_code}")
resultado = response.json()
print(f"Predicciones totales: {resultado['estadisticas_uso']['total_predicciones']}")
print(f"Métricas del modelo:")
for metrica, valor in resultado['metricas_entrenamiento'].items():
    print(f"  - {metrica.upper()}: {valor}")
print()

# 6. Test de validación de datos para reentrenamiento
print("6️⃣ POST /reentrenar/validar (Validar datos)")
print("-" * 40)
datos_validacion = {
    "datos": datos_lote,
    "precios": [180000.0, 250000.0, 320000.0],
    "epochs": 50
}
response = requests.post(f"{BASE_URL}/reentrenar/validar", json=datos_validacion)
print(f"Status Code: {response.status_code}")
print(f"Respuesta: {json.dumps(response.json(), indent=2)}")

print("\n" + "="*60)
print("✓ TODOS LOS TESTS COMPLETADOS")
print("="*60)

### 10.2 Testing con cURL (desde terminal)

También puedes probar la API usando cURL desde la terminal:

```bash
# GET /
curl http://127.0.0.1:8000/

# POST /predecir
curl -X POST "http://127.0.0.1:8000/predecir" \
  -H "Content-Type: application/json" \
  -d '{"tamano": 120, "habitaciones": 3, "banos": 2, "antiguedad": 5}'

# GET /monitorizar
curl http://127.0.0.1:8000/monitorizar
```

---

## 11. Exportar la API a un Archivo Python Standalone

Para usar la API en producción, es mejor tenerla en un archivo `.py` independiente.
Esta celda exporta todo el código necesario.

In [ ]:
# Exportar la API a un archivo Python standalone
codigo_api = '''
#!/usr/bin/env python3
"""
API REST para Predicción de Precios de Casas usando FastAPI
Generado desde el tutorial de Jupyter Notebook
"""

# Importaciones
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
import uvicorn
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os
import json
from datetime import datetime
from typing import List
import pickle

# Variables globales
modelo_actual = None
scaler_actual = None
metadata_actual = {}
historial_predicciones = []

# [AQUÍ IRÍA TODO EL CÓDIGO DE LOS MODELOS PYDANTIC Y ENDPOINTS]
# Por brevedad, este es un esqueleto. En la práctica, copiarías todo el código anterior.

# Función principal
if __name__ == "__main__":
    # Cargar el modelo al inicio
    cargar_modelo()
    
    # Iniciar servidor
    uvicorn.run(
        app,
        host="0.0.0.0",
        port=8000,
        log_level="info"
    )
'''

# Guardar el archivo
with open('api_precios_casas.py', 'w', encoding='utf-8') as f:
    f.write(codigo_api)

print("✓ Archivo 'api_precios_casas.py' exportado")
print("\nPara ejecutarlo:")
print("  python api_precios_casas.py")
print("\nO con uvicorn:")
print("  uvicorn api_precios_casas:app --reload")

---

## 12. Resumen y Mejores Prácticas

### ✅ Lo que hemos aprendido:

1. **Guardado de modelos**: Diferencias entre `.keras` y `.h5`
2. **FastAPI básico**: Crear endpoints REST
3. **Validación con Pydantic**: Asegurar datos correctos
4. **Predicciones**: Individual y por lotes
5. **Monitorización**: Tracking de métricas y uso
6. **Reentrenamiento**: Actualizar modelos sin reconstruir la API

### 🎯 Mejores prácticas para producción:

1. **Seguridad**:
   - Añadir autenticación (OAuth2, API Keys)
   - Usar HTTPS (TLS/SSL)
   - Limitar rate limiting (evitar abuso)
   - Validar TODOS los inputs

2. **Escalabilidad**:
   - Usar workers múltiples (`uvicorn --workers 4`)
   - Implementar caché (Redis)
   - Load balancing (Nginx)
   - Contenedores Docker

3. **Monitorización**:
   - Logging estructurado
   - Métricas con Prometheus
   - Alertas automáticas
   - Tracking de performance

4. **Datos y Modelos**:
   - Versionado de modelos (MLflow, DVC)
   - A/B testing de modelos
   - Validación de datos (Great Expectations)
   - Backup automático

5. **Código**:
   - Tests unitarios (pytest)
   - Tests de integración
   - CI/CD (GitHub Actions, GitLab CI)
   - Documentación completa

### 📚 Recursos adicionales:

- FastAPI: https://fastapi.tiangolo.com/
- TensorFlow/Keras: https://www.tensorflow.org/
- Pydantic: https://pydantic-docs.helpmanual.io/
- MLOps: https://ml-ops.org/

---

## 🎓 Ejercicios propuestos:

1. **Básico**: Añadir un endpoint para eliminar el historial de predicciones
2. **Intermedio**: Implementar autenticación con API Key
3. **Avanzado**: Crear un sistema de versionado de modelos (cargar diferentes versiones)
4. **Experto**: Implementar A/B testing entre dos modelos diferentes

---

**¡Gracias por completar este tutorial!** 🚀

Si tienes preguntas o sugerencias, no dudes en consultarlas.